[GoogleTest]: https://github.com/google/googletest
[CMake]: https://cmake.org/
# [GoogleTest][] による UnitTest

[CMake][]によって、Build&Test環境の準備が出来たところで、
[GoogleTest][]によって UnitTest を作成&実行します。

### References
[GoogleTest][]についての使い方は以下を読むと大抵は分かると思います。
* [Google Test ドキュメント日本語訳 - 入門ガイド](http://opencv.jp/googletestdocs/primer.html#)
* [Google Test ドキュメント日本語訳 - 上級ガイド](http://opencv.jp/googletestdocs/advancedguide.html#advancedguide)


# UnitTest の作成

ここで UnitTest 用のテストコードファイルの構成を示します。

```cpp
#include "gtest/gtest.h"
#include "gmock/gmock.h"

/// テストコード用に include が必要であれば、
/// この辺に書いておきます。

// ::testing::Testクラスからの派生クラスとしてのテスト用クラスを定義します。
class Test_hoge: public ::testing::Test {
// フィクスチャの派生クラスからフィクスチャメンバにアクセスしたいため、
// protected でメンバを宣言します。
protected:
    virtual void SetUp() {
        // SetUp() はテストごとにオブジェクトの準備をするためのものです。
        hoge_initialize();
    }
    virtual void TearDown() {
        // TearDown() は SetUp()で確保されたリソースを解放するためのもので、
        // SetUp() と同様にテストごとに実行されます。
        hoge_finalize();
    }
    
    // ここに必要ならば、複数のテストで共有するサブルーチンやインスタンス変数などを書きます。
    void *hoge;
    void *fuga;
    
    void hoge_calloc() {
        // hoge hoge
        hoge = calloc(1, sizeof(int));
    }
    void fuga_calloc() {
        // fuga fuga
        fuga = calloc(1, sizeof(int));
    }   
};

// ここで、テスト関数で利用する Assertion の宣言を行っておきます。
// いろんな種類の Assertion があるので、検査したいことに一致するものがあるかどうか
// ガイドを読むことで確認できます。
using ::testing::Eq;

// TEST_F()でテスト関数を作成します。
TEST_F(Test_hoge/*上で定義したテスト用クラスを書きます*/, hogehoge/*任意のテスト名です*/)
{
    // 実行前に SetUp() が実行されます。
    int a = 0;
    ASSERT_EQ(0, a);
    // 実行後に TearDown() が実行されます。
}

// 複数テストを実施したい場合は、複数のテスト関数を作成します。
TEST_F(Test_hoge, fugafuga)
{
    // 実行前に SetUp() が実行されます。
    
    // Setup(), TearDown()はテスト関数ごとに実行されます。
    int b = 0;
    ASSERT_EQ(0, b);
    
    // 実行後に TearDown() が実行されます。
}

// gtest_main ライブラリを link していれば、main()関数を用意する必要はありません。
```


# UnitTest の実行
UnitTest の実行には`CTest`が使えます。

```bash
$ pushd build
$ cmake ..
$ make
$ ctest -V
```

`ctest- V`の実行結果は以下のように、テスト結果と実行時間などが表示されます。
```
1: [==========] Running 1 test from 1 test case.
1: [----------] Global test environment set-up.
1: [----------] 1 test from Test_hoge
1: [ RUN      ] Test_hoge.new_destroy
1: [       OK ] Test_hoge.new_destroy (0 ms)
1: [----------] 1 test from Test_hoge (0 ms total)
1: 
1: [----------] Global test environment tear-down
1: [==========] 1 test from 1 test case ran. (0 ms total)
1: [  PASSED  ] 1 test.
1/1 Test #1: Test_hoge ........................   Passed    0.00 sec

100% tests passed, 0 tests failed out of 1

Label Time Summary:
Test_hoge    =   0.00 sec*proc (1 test)

Total Test time (real) =   0.01 sec
```


# UnitTest の実行 (cont'd)
また、特定のテストを実行したい場合は、
以下のように`-L`オプションで先のテスト用クラスを指定すれば、指定のテストのみを実行できます。

```bash
$ ctest -V -L <RegularExpression>
```

e.g. 末尾が `_hoge` で終わるテスト用クラスのみ、テストを実行する。
```bash
$ ctest -V -L _hoge$
```
